## Environment

In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/notebooks'):
    main_path = p[:-len('/notebooks')]
if sys.path[0].endswith('/techdoc/content'):
    main_path = p[:-len('/techdoc/content')]
    
# Windows OS
if sys.path[0].endswith('\\notebooks'): 
    main_path = p[:-len('\\notebooks')]
if sys.path[0].endswith('\\techdoc\content'): 
    main_path = p[:-len('\\techdoc\content')]

sys.path[0] = main_path

In [3]:
import os, gc, itertools
from termcolor import colored
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from scipy.signal import find_peaks
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import auc

from src import (config, describe_data, features,
                 preprocess, training)

/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Data Load

In [4]:
# ===== LOAD DATA ======
metadata = pd.read_csv(config.DATA_DIR + 'metadata.csv')
print(f'Metadata: {metadata.shape}')

train_labels = pd.read_csv(config.DATA_DIR + 'train_labels.csv')
print(f'Train labels: {train_labels.shape}')

valid_labels = pd.read_csv(config.DATA_DIR + 'val_labels.csv')
print(f'Train labels: {valid_labels.shape}')

submission = pd.read_csv(config.DATA_DIR + 'submission_format.csv')
print(f'Submission: {submission.shape}')

Metadata: (1570, 5)
Train labels: (766, 11)
Train labels: (293, 11)
Submission: (804, 11)


In [5]:
# ===== MODEL SAMPLES ======
train = metadata[metadata.split == 'train'].copy().reset_index(drop=True)
print(f'TRAIN: {train.shape}')

valid = metadata[metadata.split == 'val'].copy().reset_index(drop=True)
print(f'VALID: {valid.shape}')

test = metadata[metadata.split == 'test'].copy().reset_index(drop=True)
print(f'TEST: {test.shape}')

TRAIN: (766, 5)
VALID: (293, 5)
TEST: (511, 5)


In [6]:
# ===== FILE PATHS OF SAMPLES =====
train_files = metadata[metadata.split == 'train']['features_path'].to_dict()
valid_files = metadata[metadata.split == 'val']['features_path'].to_dict()
test_files = metadata[metadata.split == 'test']['features_path'].to_dict()
# Train & Valid files
trva_files = train_files.copy()
trva_files.update(valid_files)
# All files
all_test_files = valid_files.copy()
all_test_files.update(test_files)

# Ion type list
ion_list = list(np.arange(0,100,1.0))
ion_list.remove(4.0)

# Get the names of the target columns in a list
target_labels_list = [i for i in train_labels.columns if i not in ['sample_id']]
print(target_labels_list)

['basalt', 'carbonate', 'chloride', 'iron_oxide', 'oxalate', 'oxychlorine', 'phyllosilicate', 'silicate', 'sulfate', 'sulfide']


# Notes

- How long does it take for temperature to rise to certain level where we get abundance?
- How to account for the non-linear diferences of time vs temp?
- Max value of abundance of each ion - take temp reading and time - or duration
- How to define characteristic of peaks in ion abundances; temp at which they occur, the shape of peaks (height, width, area)
- How to deal with overlapping peaks
- Size of the peaks

**Compute min and max temp for all available samples, including the ion types**

In [ ]:
# Get min/max temperature values and unique ion list
# Preprocess the data to remove noise before calculation
min_temp, max_temp, ion_list = preprocess.compute_min_max_temp_ion(metadata)
print(colored(f'Min temp = {min_temp}; Max temp = {max_temp}', 'blue'))
print(colored(f'Number of unqiue ions: {len(ion_list)}', 'blue'))

100%|██████████| 1570/1570 [01:53<00:00, 13.80it/s]

Min temp = -63.563; Max temp = 1484.13
Number of unqiue ions: 99


# SAMPLES

## `metadata` with top 3 ions
- not normalized since it messes up the target encoding later on
    - *if used as a feature normalize prior to training*

In [44]:
metadata = pd.read_csv(config.DATA_DIR + 'metadata.csv')
topN_ions = features.get_topN_ions(metadata, N=3, normalize=False)
topN_ions.index = topN_ions.index.set_names('sample_id')
topN_ions = topN_ions.reset_index()
topN_ions.head()

100%|██████████| 1570/1570 [01:55<00:00, 13.55it/s]


,sample_id,top_1,top_2,top_3
0,S0000,18.0,17.0,16.0
1,S0001,44.0,16.0,2.0
2,S0002,18.0,32.0,17.0
3,S0003,18.0,44.0,17.0
4,S0004,18.0,32.0,17.0


In [45]:
# Merge to metadata
metadata = pd.merge(metadata, topN_ions, on='sample_id', how='left')
metadata[[i for i in metadata if i.startswith('top_')]] = metadata[[i for i in metadata if i.startswith('top_')]].astype('int')
print(metadata.shape)
print(metadata.head(2))

metadata.to_csv(os.path.join(config.DATA_DIR_OUT + 'metadata.csv'), 
                index=False)

(1570, 8)
  sample_id  split instrument_type             features_path  \
0     S0000  train      commercial  train_features/S0000.csv   
1     S0001  train      commercial  train_features/S0001.csv   
2     S0002  train      commercial  train_features/S0002.csv   
3     S0003  train      commercial  train_features/S0003.csv   
4     S0004  train      commercial  train_features/S0004.csv   

                  features_md5_hash  top_1  top_2  top_3  
0  017b9a71a702e81a828e6242aa15f049     18     17     16  
1  0d09840214054d254bd49436c6a6f315     44     16      2  
2  3f58b3c9b001bfed6ed4e4f757083e09     18     32     17  
3  e9a12f96114a2fda60b36f4c0f513fb1     18     44     17  
4  b67603d3931897bfa796ac42cc16de78     18     32     17  


## `fts_maxrelabund_tempion` - Max relative abundance per temp bin and ion type

Bin the temp from min to max in intervals of 100 degrees and combine with the ion type i.e. `m/z`. For each combination compute the maximum relative abundance.

In [7]:
# ===== TRAIN SAMPLE =====
print(f'Number of files: {len(train_files)}')
fts_maxrelabund_tempion = features.features_iontemp_abun(metadata, train_files, 
                                                         detrend_method='min')
print(f'Shape: {fts_maxrelabund_tempion.shape}')
assert fts_maxrelabund_tempion.shape[0] == len(train_files)
fts_maxrelabund_tempion = fts_maxrelabund_tempion.replace(np.nan, 0)
fts_maxrelabund_tempion.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion.csv'), 
                               index=False)

"""
fts_maxrelabund_tempion = features.features_iontemp_abun(metadata, train_files, 
                                                         detrend_method='lin_reg')
print(f'Shape: {fts_maxrelabund_tempion.shape}')
assert fts_maxrelabund_tempion.shape[0] == len(train_files)
fts_maxrelabund_tempion = fts_maxrelabund_tempion.replace(np.nan, 0)
fts_maxrelabund_tempion.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_lr.csv'), 
                               index=False)

fts_maxrelabund_tempion = features.features_iontemp_abun(metadata, train_files, 
                                                         detrend_method='poly')
print(f'Shape: {fts_maxrelabund_tempion.shape}')
assert fts_maxrelabund_tempion.shape[0] == len(train_files)
fts_maxrelabund_tempion = fts_maxrelabund_tempion.replace(np.nan, 0)
fts_maxrelabund_tempion.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_poly.csv'), 
                               index=False)
"""

Number of files: 766
Number of samples: 766
Shape: (766, 1584)


"\nfts_maxrelabund_tempion = features.features_iontemp_abun(metadata, train_files, \n                                                         detrend_method='lin_reg')\nprint(f'Shape: {fts_maxrelabund_tempion.shape}')\nassert fts_maxrelabund_tempion.shape[0] == len(train_files)\nfts_maxrelabund_tempion = fts_maxrelabund_tempion.replace(np.nan, 0)\nfts_maxrelabund_tempion.to_csv(os.path.join(config.DATA_DIR_OUT + \n                                            'fts_maxrelabund_tempion_lr.csv'), \n                               index=False)\n\nfts_maxrelabund_tempion = features.features_iontemp_abun(metadata, train_files, \n                                                         detrend_method='poly')\nprint(f'Shape: {fts_maxrelabund_tempion.shape}')\nassert fts_maxrelabund_tempion.shape[0] == len(train_files)\nfts_maxrelabund_tempion = fts_maxrelabund_tempion.replace(np.nan, 0)\nfts_maxrelabund_tempion.to_csv(os.path.join(config.DATA_DIR_OUT + \n                                          

In [8]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
print(f'Number of files: {len(trva_files)}')
fts_maxrelabund_tempion_trvl = features.features_iontemp_abun(metadata, trva_files, 
                                                              detrend_method='min')
print(f'Shape: {fts_maxrelabund_tempion_trvl.shape}')
assert fts_maxrelabund_tempion_trvl.shape[0] == len(trva_files)
fts_maxrelabund_tempion_trvl = fts_maxrelabund_tempion_trvl.replace(np.nan, 0)
fts_maxrelabund_tempion_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_trvl.csv'), 
                index=False)

"""
fts_maxrelabund_tempion_trvl = features.features_iontemp_abun(metadata, trva_files, 
                                                              detrend_method='lin_reg')
print(f'Shape: {fts_maxrelabund_tempion_trvl.shape}')
assert fts_maxrelabund_tempion_trvl.shape[0] == len(trva_files)
fts_maxrelabund_tempion_trvl = fts_maxrelabund_tempion_trvl.replace(np.nan, 0)
fts_maxrelabund_tempion_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_trvl_lr.csv'), 
                index=False)

print(f'Number of all test files: {len(trva_files)}')
fts_maxrelabund_tempion_trvl = features.features_iontemp_abun(metadata, trva_files, 
                                                              detrend_method='poly')
print(f'Shape: {fts_maxrelabund_tempion_trvl.shape}')
assert fts_maxrelabund_tempion_trvl.shape[0] == len(trva_files)
fts_maxrelabund_tempion_trvl = fts_maxrelabund_tempion_trvl.replace(np.nan, 0)
fts_maxrelabund_tempion_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_trvl_poly.csv'), 
                index=False)
"""

Number of files: 1059
Number of samples: 1059
Shape: (1059, 1584)


"\nfts_maxrelabund_tempion_trvl = features.features_iontemp_abun(metadata, trva_files, \n                                                              detrend_method='lin_reg')\nprint(f'Shape: {fts_maxrelabund_tempion_trvl.shape}')\nassert fts_maxrelabund_tempion_trvl.shape[0] == len(trva_files)\nfts_maxrelabund_tempion_trvl = fts_maxrelabund_tempion_trvl.replace(np.nan, 0)\nfts_maxrelabund_tempion_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + \n                                            'fts_maxrelabund_tempion_trvl_lr.csv'), \n                index=False)\n\nprint(f'Number of all test files: {len(trva_files)}')\nfts_maxrelabund_tempion_trvl = features.features_iontemp_abun(metadata, trva_files, \n                                                              detrend_method='poly')\nprint(f'Shape: {fts_maxrelabund_tempion_trvl.shape}')\nassert fts_maxrelabund_tempion_trvl.shape[0] == len(trva_files)\nfts_maxrelabund_tempion_trvl = fts_maxrelabund_tempion_trvl.replace(np.nan, 0)\nfts_

In [9]:
# ===== VALID & TEST SAMPLE =====
print(f'Number of all test files: {len(all_test_files)}')
fts_maxrelabund_tempion_VT = features.features_iontemp_abun(metadata, all_test_files, 
                                                            detrend_method='min')
print(f'Shape: {fts_maxrelabund_tempion_VT.shape}')
assert fts_maxrelabund_tempion_VT.shape[0] == len(all_test_files)
fts_maxrelabund_tempion_VT = fts_maxrelabund_tempion_VT.replace(np.nan,0)
fts_maxrelabund_tempion_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_VT.csv'),
                index=False)

"""
fts_maxrelabund_tempion_VT = features.features_iontemp_abun(metadata, all_test_files, 
                                                            detrend_method='lin_reg')
print(f'Shape: {fts_maxrelabund_tempion_VT.shape}')
assert fts_maxrelabund_tempion_VT.shape[0] == len(all_test_files)
fts_maxrelabund_tempion_VT = fts_maxrelabund_tempion_VT.replace(np.nan,0)
fts_maxrelabund_tempion_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_VT_lr.csv'),
                index=False)

fts_maxrelabund_tempion_VT = features.features_iontemp_abun(metadata, all_test_files, 
                                                            detrend_method='poly')
print(f'Shape: {fts_maxrelabund_tempion_VT.shape}')
assert fts_maxrelabund_tempion_VT.shape[0] == len(all_test_files)
fts_maxrelabund_tempion_VT = fts_maxrelabund_tempion_VT.replace(np.nan,0)
fts_maxrelabund_tempion_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_maxrelabund_tempion_VT_poly.csv'),
                index=False)
"""

Number of all test files: 804
Number of samples: 804
Shape: (804, 1584)


"\nfts_maxrelabund_tempion_VT = features.features_iontemp_abun(metadata, all_test_files, \n                                                            detrend_method='lin_reg')\nprint(f'Shape: {fts_maxrelabund_tempion_VT.shape}')\nassert fts_maxrelabund_tempion_VT.shape[0] == len(all_test_files)\nfts_maxrelabund_tempion_VT = fts_maxrelabund_tempion_VT.replace(np.nan,0)\nfts_maxrelabund_tempion_VT.to_csv(os.path.join(config.DATA_DIR_OUT + \n                                            'fts_maxrelabund_tempion_VT_lr.csv'),\n                index=False)\n\nfts_maxrelabund_tempion_VT = features.features_iontemp_abun(metadata, all_test_files, \n                                                            detrend_method='poly')\nprint(f'Shape: {fts_maxrelabund_tempion_VT.shape}')\nassert fts_maxrelabund_tempion_VT.shape[0] == len(all_test_files)\nfts_maxrelabund_tempion_VT = fts_maxrelabund_tempion_VT.replace(np.nan,0)\nfts_maxrelabund_tempion_VT.to_csv(os.path.join(config.DATA_DIR_OUT + \n   

## `fts_tempion_area` - area of the abundance curve per temp bin and ion

In [33]:
# 
# ===== TRAIN SAMPLE =====
print(f'Number of files: {len(train_files)}')
fts_tempion_area = features.features_iontemp_area(metadata, train_files, 
                                                         detrend_method='min')
print(f'Shape: {fts_tempion_area.shape}')
assert fts_tempion_area.shape[0] == len(train_files)
fts_tempion_area = fts_tempion_area.replace(np.nan, 0)
fts_tempion_area.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_tempion_area.csv'), index=False)

Number of files: 766
Number of samples: 766
Sample: 0


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 1


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 2


...

Sample: 764


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 765


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Shape: (766, 1583)


In [37]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
print(f'Number of files: {len(trva_files)}')
fts_tempion_area_trvl = features.features_iontemp_area(metadata, trva_files, 
                                                              detrend_method='min')
print(f'Shape: {fts_tempion_area_trvl.shape}')
assert fts_tempion_area_trvl.shape[0] == len(trva_files)
fts_tempion_area_trvl = fts_tempion_area_trvl.replace(np.nan, 0)
fts_tempion_area_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_tempion_area_trvl.csv'), index=False)


Number of files: 1059
Number of samples: 1059
Sample: 0


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 1


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 2


...

Sample: 1057


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 1058


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Shape: (1059, 1583)


In [36]:
# ===== VALID & TEST SAMPLE =====
print(f'Number of all test files: {len(all_test_files)}')
fts_tempion_area_VT = features.features_iontemp_area(metadata, all_test_files, 
                                                            detrend_method='min')
print(f'Shape: {fts_tempion_area_VT.shape}')
assert fts_tempion_area_VT.shape[0] == len(all_test_files)
fts_tempion_area_VT = fts_tempion_area_VT.replace(np.nan,0)
fts_tempion_area_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                            'fts_tempion_area_VT.csv'), index=False)

Number of all test files: 804
Number of samples: 804
Sample: 766


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 767


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 768


...

Sample: 1568


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Sample: 1569


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


Shape: (804, 1583)


## `fts_tempion_area_filtered`- Remove insignificatn m/z ions

In [51]:
# ===== TRAIN & VALID & TEST SAMPLE =====
all_files = train_files.copy()
all_files.update(valid_files)
all_files.update(test_files)

print(f'Number of all test files: {len(all_files)}')
fts_tempion_area_filtered_ALL = features.features_iontemp_area(metadata, all_files, 
                                                            detrend_method='min',
                                                            filter_ions=True)
print(f'Shape: {fts_tempion_area_filtered_ALL.shape}')
assert fts_tempion_area_filtered_ALL.shape[0] == len(all_files)
fts_tempion_area_filtered_ALL = fts_tempion_area_filtered_ALL.replace(np.nan,0)
fts_tempion_area_filtered_ALL.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                                  'fts_tempion_area_filtered_ALL.csv'), index=False)

Number of all test files: 1570


/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
/Users/itacdonev/opt/miniconda3/envs/nasamars/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index con

Shape: (1570, 1260)


In [59]:
# ===== TRAIN SAMPLE =====
fts_tempion_area_filtered = fts_tempion_area_filtered_ALL.iloc[:train_labels.shape[0],:].copy()
assert fts_tempion_area_filtered.shape[0] == len(train_files)
print(fts_tempion_area_filtered.shape)
fts_tempion_area_filtered.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                              'fts_tempion_area_filtered.csv'), index=False)

# ===== TRAIN & VALID SAMPLE =====
fts_tempion_area_filtered_trvl = fts_tempion_area_filtered_ALL.iloc[:len(trva_files),:].copy()
assert fts_tempion_area_filtered_trvl.shape[0] == len(trva_files)
print(fts_tempion_area_filtered_trvl.shape)
fts_tempion_area_filtered_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                                 'fts_tempion_area_filtered_trvl.csv'), index=False)

# ===== VALID & TEST SAMPLE =====
fts_tempion_area_filtered_VT = fts_tempion_area_filtered_ALL.iloc[:len(all_test_files),:].copy()
assert fts_tempion_area_filtered_VT.shape[0] == len(all_test_files)
print(fts_tempion_area_filtered_VT.shape)
fts_tempion_area_filtered_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                                 'fts_tempion_area_filtered_VT.csv'), index=False)

(766, 1260)
(1059, 1260)
(804, 1260)


## `fts_ion_peaks` - peak analysis and stats

In [10]:
# ===== TRAIN SAMPLE =====
fts_ion_peaks = features.features_ion_peaks(train_files, metadata, ion_list, detrend_method='min')
print(fts_ion_peaks.shape)
fts_ion_peaks.head()
fts_ion_peaks.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                       'fts_ion_peaks.csv'),
                index=False)

100%|██████████| 766/766 [02:20<00:00,  5.46it/s]


(766, 396)


In [11]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
fts_ion_peaks_trvl = features.features_ion_peaks(trva_files, metadata, ion_list, detrend_method='min')
print(fts_ion_peaks_trvl.shape)
fts_ion_peaks_trvl.head()
fts_ion_peaks_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                       'fts_ion_peaks_trvl.csv'),
                index=False)

100%|██████████| 1059/1059 [03:13<00:00,  5.48it/s]


(1059, 396)


In [12]:
# ===== VALID & TEST SAMPLE (_VT) =====
fts_ion_peaks_VT = features.features_ion_peaks(all_test_files, metadata, ion_list, detrend_method='min')
print(fts_ion_peaks_VT.shape)
fts_ion_peaks_VT.head()
fts_ion_peaks_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                       'fts_ion_peaks_VT.csv'),
                index=False)

100%|██████████| 804/804 [02:37<00:00,  5.12it/s]


(804, 396)


## `combo_maxabund_peaks` - Combine max rel abund per temp_ion bin and peak analysis

In [13]:
# ===== TRAIN SAMPLE =====
combo_maxabund_peaks = pd.concat([fts_maxrelabund_tempion, fts_ion_peaks], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_peaks.shape)
combo_maxabund_peaks.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks.csv'), 
                                  index=False)

(766, 1980)


In [20]:
combo_maxabund_peaks.head()

,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_90.0,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
combo_maxabund_peaks_trvl = pd.concat([fts_maxrelabund_tempion_trvl, fts_ion_peaks_trvl], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_peaks_trvl.shape)
combo_maxabund_peaks_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_trvl.csv'), 
                                  index=False)

(1059, 1980)


In [21]:
combo_maxabund_peaks_trvl.head()


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_90.0,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# ===== VALID & TEST SAMPLE =====
combo_maxabund_peaks_VT = pd.concat([fts_maxrelabund_tempion_VT, fts_ion_peaks_VT], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_peaks_VT.shape)
combo_maxabund_peaks_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_VT.csv'), 
                                  index=False)

(804, 1980)


In [22]:
combo_maxabund_peaks_VT.head()


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_90.0,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0
0,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.029256,0.040753,0.037022,0.041347,0.035408,0.036785,0.033979,0.023970,0.023692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000261,0.000842,0.001313,0.001544,0.001405,0.002993,0.001424,0.000768,0.000992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.003460,0.003929,0.003286,0.002893,0.010054,0.003877,0.001918,0.000824,0.000323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `combo_maxabund_peaks_slope`

In [23]:
# ===== TRAIN SAMPLE =====
# Dict with sample_id as keys
fts_slope_timetemp = features.slope_time_temp(train_files, metadata, detrend_method='min')
combo_maxabund_peaks_slope = combo_maxabund_peaks.copy()
combo_maxabund_peaks_slope['slope_tt'] = combo_maxabund_peaks_slope.index.map(fts_slope_timetemp)
combo_maxabund_peaks_slope.to_csv(os.path.join(config.DATA_DIR_OUT + 'combo_maxabund_peaks_slope.csv'), 
                                  index=False)
combo_maxabund_peaks_slope.head()

100%|██████████| 766/766 [00:55<00:00, 13.92it/s]


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [24]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
fts_slope_timetemp_trvl = features.slope_time_temp(trva_files, metadata, detrend_method='min')
combo_maxabund_peaks_slope_trvl = combo_maxabund_peaks_trvl.copy()
combo_maxabund_peaks_slope_trvl['slope_tt'] = combo_maxabund_peaks_slope_trvl.index.map(fts_slope_timetemp_trvl)
combo_maxabund_peaks_slope_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 'combo_maxabund_peaks_slope_trvl.csv'), 
                                  index=False)
combo_maxabund_peaks_slope_trvl.head()

100%|██████████| 1059/1059 [01:24<00:00, 12.54it/s]


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [25]:
# ===== VALID & TEST SAMPLE (_VT) =====
fts_slope_timetemp_VT = features.slope_time_temp(all_test_files, metadata, detrend_method='min')
combo_maxabund_peaks_slope_VT = combo_maxabund_peaks_VT.copy()
combo_maxabund_peaks_slope_VT['slope_tt'] = combo_maxabund_peaks_slope_VT.index.map(fts_slope_timetemp_VT)
combo_maxabund_peaks_slope_VT.to_csv(os.path.join(config.DATA_DIR_OUT +
                                            'combo_maxabund_peaks_slope_VT.csv'),
                index=False)
combo_maxabund_peaks_slope_VT.head()

100%|██████████| 804/804 [01:04<00:00, 12.37it/s]


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt
0,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,0.0,0.029256,0.040753,0.037022,0.041347,0.035408,0.036785,0.033979,0.023970,0.023692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.000261,0.000842,0.001313,0.001544,0.001405,0.002993,0.001424,0.000768,0.000992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,0.0,0.003460,0.003929,0.003286,0.002893,0.010054,0.003877,0.001918,0.000824,0.000323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


## `combo_maxabund_peaks_slope_topions`

(temp bins & ions at max abundance) + (#peaks, time to, temp at, abund at) + slope + (top 3 ions)

In [46]:
#TODO Normalize topions before training - integers now!!!!
# ===== TRAIN SAMPLE =====
metadata = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'metadata.csv'))
metadata_upt = metadata.iloc[:train.shape[0],:].copy()
combo_maxabund_peaks_slope = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope.csv'))
assert metadata_upt.shape[0] == combo_maxabund_peaks_slope.shape[0]
print(f'Shape: {metadata_upt.shape}')

combo_maxabund_peaks_slope_topions = combo_maxabund_peaks_slope.copy()
fts = [i for i in metadata_upt if i.startswith('top_')]
combo_maxabund_peaks_slope_topions = pd.concat([combo_maxabund_peaks_slope_topions, 
                                                metadata_upt[fts]],
                                               axis=1)
combo_maxabund_peaks_slope_topions.to_csv(os.path.join(config.DATA_DIR_OUT +
                                            'combo_maxabund_peaks_slope_topions.csv'),
                index=False)
print(f'Sample shape: {combo_maxabund_peaks_slope_topions.shape}')
combo_maxabund_peaks_slope_topions.head(2)

Shape: (766, 8)
Sample shape: (766, 1984)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt,top_1,top_2,top_3
0,0.0,0.004085,0.004641,0.001394,0.000188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.195083,18,17,16
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.167294,44,16,2


In [47]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
metadata = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'metadata.csv'))
metadata_upt = metadata.iloc[:len(trva_files),:].copy()
combo_maxabund_peaks_slope_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope_trvl.csv'))
assert metadata_upt.shape[0] == combo_maxabund_peaks_slope_trvl.shape[0]
print(f'Shape: {metadata_upt.shape}')

combo_maxabund_peaks_slope_topions_trvl = combo_maxabund_peaks_slope_trvl.copy()
fts = [i for i in metadata_upt if i.startswith('top_')]
combo_maxabund_peaks_slope_topions_trvl = pd.concat([combo_maxabund_peaks_slope_topions_trvl, 
                                                metadata_upt[fts]],
                                               axis=1)
combo_maxabund_peaks_slope_topions_trvl.to_csv(os.path.join(config.DATA_DIR_OUT +
                                            'combo_maxabund_peaks_slope_topions_trvl.csv'),
                index=False)
print(f'Sample shape: {combo_maxabund_peaks_slope_topions_trvl.shape}')
combo_maxabund_peaks_slope_topions_trvl.head(2)

Shape: (1059, 8)
Sample shape: (1059, 1984)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt,top_1,top_2,top_3
0,0.0,0.004085,0.004641,0.001394,0.000188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.195083,18,17,16
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.167294,44,16,2


In [48]:
# ===== VALID & TEST SAMPLE (_VT) =====
metadata_upt = metadata.iloc[train.shape[0]:,:].copy()
combo_maxabund_peaks_VT = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope_VT.csv'))
assert metadata_upt.shape[0] == combo_maxabund_peaks_VT.shape[0]
print(f'Shape: {metadata_upt.shape}')

combo_maxabund_peaks_slope_topions_VT = combo_maxabund_peaks_VT.copy()
fts = [i for i in metadata_upt if i.startswith('top_')]
combo_maxabund_peaks_slope_topions_VT = pd.concat([combo_maxabund_peaks_slope_topions_VT, 
                                                metadata_upt[fts].reset_index(drop=True)],
                                               axis=1)
assert combo_maxabund_peaks_slope_topions_VT.shape[0]== len(all_test_files)
print(f'Sample shape: {combo_maxabund_peaks_slope_topions_VT.shape}')
# Save the sample
combo_maxabund_peaks_slope_topions_VT.to_csv(os.path.join(config.DATA_DIR_OUT +
                                            'combo_maxabund_peaks_slope_topions_VT.csv'),
                index=False)
combo_maxabund_peaks_slope_topions_VT.head(2)

Shape: (804, 8)
Sample shape: (804, 1984)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt,top_1,top_2,top_3
0,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.609997,18,1,0
1,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,0.0,0.0,0.0,0.0,0.0,0.0,0.608090,18,17,32


## `combo_maxabund_peaks_slope_area`

In [45]:
# ===== TRAIN SAMPLE =====
df_fts = features.features_area(train_files, metadata, 'min')
combo_maxabund_peaks_slope = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope.csv'))
combo_maxabund_peaks_slope_area = combo_maxabund_peaks_slope.copy()
combo_maxabund_peaks_slope_area['area'] = combo_maxabund_peaks_slope_area.index.map(df_fts)
combo_maxabund_peaks_slope_area.to_csv(os.path.join(config.DATA_DIR_OUT +
                                   'combo_maxabund_peaks_slope_area.csv'),
                index=False)
print(combo_maxabund_peaks_slope_area.shape)
combo_maxabund_peaks_slope_area.head(2)

(766, 1982)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt,area
0,0.0,0.004085,0.004641,0.001394,0.000188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.195083,147.838057
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.167294,70.159331


In [46]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
df_fts = features.features_area(trva_files, metadata, 'min')
combo_maxabund_peaks_slope_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 
                                                      'combo_maxabund_peaks_slope_trvl.csv'))
combo_maxabund_peaks_slope_area_trvl = combo_maxabund_peaks_slope_trvl.copy()
combo_maxabund_peaks_slope_area_trvl['area'] = combo_maxabund_peaks_slope_area_trvl.index.map(df_fts)
combo_maxabund_peaks_slope_area_trvl.to_csv(os.path.join(config.DATA_DIR_OUT +
                                   'combo_maxabund_peaks_slope_area_trvl.csv'),
                index=False)
print(combo_maxabund_peaks_slope_area_trvl.shape)
combo_maxabund_peaks_slope_area_trvl.head(2)

100%|██████████| 1059/1059 [01:16<00:00, 13.92it/s]


(1059, 1982)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt,area
0,0.0,0.004085,0.004641,0.001394,0.000188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.195083,147.838057
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.167294,70.159331


In [52]:
# ===== VALID & TEST SAMPLE (_VT) =====
#df_fts = features.features_area(all_test_files, metadata, 'min')
combo_maxabund_peaks_slope_VT = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 
                                                      'combo_maxabund_peaks_slope_VT.csv'))
combo_maxabund_peaks_slope_area_VT = combo_maxabund_peaks_slope_VT.copy()
combo_maxabund_peaks_slope_area_VT.index = range(766, 766+804,1)
combo_maxabund_peaks_slope_area_VT['area'] = combo_maxabund_peaks_slope_area_VT.index.map(df_fts)
combo_maxabund_peaks_slope_area_VT.to_csv(os.path.join(config.DATA_DIR_OUT +
                                   'combo_maxabund_peaks_slope_area_VT.csv'),
                index=False)
print(combo_maxabund_peaks_slope_area_VT.shape)
combo_maxabund_peaks_slope_area_VT.head(2)

(804, 1982)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt,area
766,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.609997,506.385863
767,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.608090,445.240708


## `combo_maxabund_peaks_slope_spectra`

In [26]:
# ===== TRAIN SAMPLE =====
fts_mass_spectra = features.features_ms(metadata, train_files, 'min')
del fts_mass_spectra['sample_id']
fts_mass_spectra = fts_mass_spectra.reset_index(drop=True)

combo_maxabund_peaks_slope = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope.csv'))
combo_maxabund_peaks_slope_spectra = pd.concat([combo_maxabund_peaks_slope, fts_mass_spectra], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_peaks_slope_spectra.shape)
combo_maxabund_peaks_slope_spectra = combo_maxabund_peaks_slope_spectra.fillna(0)
combo_maxabund_peaks_slope_spectra.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_slope_spectra.csv'), 
                                  index=False)
combo_maxabund_peaks_slope_spectra.head()

100%|██████████| 766/766 [01:12<00:00, 10.51it/s]


(766, 1991)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,M,M_mra,Mp1_mra,Mp1_isotope,Mp2_mra,Mp2_isotope,hi1,hi2,C_cnt,mi_frg_diff
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,28.0,0.010877,0.000154,1.41,0.000080,0.73,,,1.29,10.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,44.0,1.000000,0.009622,0.96,0.003249,0.32,12.0,16.0,0.87,12.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,32.0,0.294738,0.000257,0.09,0.001124,0.38,,,0.08,14.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,64.0,0.011147,0.000108,0.97,0.000520,4.67,12.0,,0.88,20.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,44.0,0.029998,0.000361,1.20,0.000124,0.41,12.0,,1.10,8.0


In [27]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
fts_mass_spectra_trvl = features.features_ms(metadata, trva_files, 'min')
del fts_mass_spectra_trvl['sample_id']
fts_mass_spectra_trvl = fts_mass_spectra_trvl.reset_index(drop=True)

combo_maxabund_peaks_slope_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope_trvl.csv'))
combo_maxabund_peaks_slope_spectra_trvl = pd.concat([combo_maxabund_peaks_slope_trvl, fts_mass_spectra_trvl], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_peaks_slope_spectra_trvl.shape)
combo_maxabund_peaks_slope_spectra_trvl = combo_maxabund_peaks_slope_spectra_trvl.fillna(0)
combo_maxabund_peaks_slope_spectra_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_slope_spectra_trvl.csv'), 
                                  index=False)
combo_maxabund_peaks_slope_spectra_trvl.head()

100%|██████████| 1059/1059 [02:30<00:00,  7.01it/s]


(1059, 1991)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,M,M_mra,Mp1_mra,Mp1_isotope,Mp2_mra,Mp2_isotope,hi1,hi2,C_cnt,mi_frg_diff
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,28.0,0.010877,0.000154,1.41,0.000080,0.73,,,1.29,10.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,44.0,1.000000,0.009622,0.96,0.003249,0.32,12.0,16.0,0.87,12.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,32.0,0.294738,0.000257,0.09,0.001124,0.38,,,0.08,14.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,64.0,0.011147,0.000108,0.97,0.000520,4.67,12.0,,0.88,20.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,44.0,0.029998,0.000361,1.20,0.000124,0.41,12.0,,1.10,8.0


In [28]:
# ===== VALID & TEST SAMPLE (_VT) =====
fts_mass_spectra_VT = features.features_ms(metadata, all_test_files, 'min')
del fts_mass_spectra_VT['sample_id']
fts_mass_spectra_VT = fts_mass_spectra_VT.reset_index(drop=True)

combo_maxabund_peaks_slope_VT = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope_VT.csv'))
combo_maxabund_peaks_slope_spectra_VT = pd.concat([combo_maxabund_peaks_slope_VT, fts_mass_spectra_VT], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_peaks_slope_spectra_VT.shape)
combo_maxabund_peaks_slope_spectra_VT = combo_maxabund_peaks_slope_spectra_VT.fillna(0)
combo_maxabund_peaks_slope_spectra_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_slope_spectra_VT.csv'), 
                                  index=False)
combo_maxabund_peaks_slope_spectra_VT.head()

100%|██████████| 804/804 [01:37<00:00,  8.26it/s]


(804, 1991)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,M,M_mra,Mp1_mra,Mp1_isotope,Mp2_mra,Mp2_isotope,hi1,hi2,C_cnt,mi_frg_diff
0,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,44.0,0.155088,0.004210,2.71,0.002622,1.69,,,2.47,4.0
1,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,44.0,0.029782,0.001263,4.24,0.000445,1.49,,,3.85,12.0
2,0.0,0.029256,0.040753,0.037022,0.041347,0.035408,0.036785,0.033979,0.023970,0.023692,...,45.0,0.012341,0.004179,33.86,0.000302,2.45,,,30.78,1.0
3,0.0,0.000261,0.000842,0.001313,0.001544,0.001405,0.002993,0.001424,0.000768,0.000992,...,64.0,0.028296,0.000312,1.10,0.001381,4.88,12.0,,1.00,16.0
4,0.0,0.003460,0.003929,0.003286,0.002893,0.010054,0.003877,0.001918,0.000824,0.000323,...,44.0,0.024179,0.000300,1.24,0.000107,0.44,12.0,,1.13,6.0


## `mz_statistics`

In [34]:
# ===== TRAIN SAMPLE =====
mz_statistics = features.features_mz_stats(metadata, train_files, 'min')
mz_statistics.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                  'mz_statistics.csv'), 
                     index=True)

In [47]:
mz_statistics.shape

(766, 495)

In [35]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
mz_statistics_trvl = features.features_mz_stats(metadata, trva_files, 'min')
mz_statistics_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                  'mz_statistics_trvl.csv'), 
                     index=False)

100%|██████████| 1059/1059 [02:11<00:00,  8.07it/s]


In [36]:
# ===== VALID & TEST SAMPLE (_VT) =====
mz_statistics_VT = features.features_mz_stats(metadata, all_test_files, 'min')
mz_statistics_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                  'mz_statistics_VT.csv'), 
                     index=True)

100%|██████████| 804/804 [01:38<00:00,  8.16it/s]


In [38]:
mz_statistics = mz_statistics.reset_index(drop=True)
mz_statistics_trvl = mz_statistics_trvl.reset_index(drop=True)
mz_statistics_VT = mz_statistics_VT.reset_index(drop=True)

## `combo_maxabund_mzstats`

In [48]:
# ===== TRAIN SAMPLE =====
fts_maxrelabund_tempion = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'fts_maxrelabund_tempion.csv'))
combo_maxabund_stats = pd.concat([fts_maxrelabund_tempion, mz_statistics], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_stats.shape)
combo_maxabund_stats = combo_maxabund_stats.fillna(0)
combo_maxabund_stats.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_stats.csv'), 
                                  index=False)
combo_maxabund_stats.head()

(766, 2079)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,skew_90.0,skew_91.0,skew_92.0,skew_93.0,skew_94.0,skew_95.0,skew_96.0,skew_97.0,skew_98.0,skew_99.0
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
fts_maxrelabund_tempion_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'fts_maxrelabund_tempion_trvl.csv'))
combo_maxabund_stats_trvl = pd.concat([fts_maxrelabund_tempion_trvl, mz_statistics_trvl], 
                               axis = 1, 
                               ignore_index=False)
print(combo_maxabund_stats_trvl.shape)
combo_maxabund_stats_trvl = combo_maxabund_stats_trvl.fillna(0)
combo_maxabund_stats_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_stats_trvl.csv'), 
                                  index=False)
combo_maxabund_stats_trvl.head()

(1059, 2079)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,skew_90.0,skew_91.0,skew_92.0,skew_93.0,skew_94.0,skew_95.0,skew_96.0,skew_97.0,skew_98.0,skew_99.0
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# ===== VALID & TEST SAMPLE (_VT) =====
fts_maxrelabund_tempion_VT = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'fts_maxrelabund_tempion_VT.csv'))
combo_maxabund_stats_VT = pd.concat([fts_maxrelabund_tempion_VT, mz_statistics_VT], 
                               axis = 1, 
                               ignore_index=False)

combo_maxabund_stats_VT = combo_maxabund_stats_VT.fillna(0)
combo_maxabund_stats_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_stats_VT.csv'), 
                                  index=False)
print(combo_maxabund_stats_VT.shape)
combo_maxabund_stats_VT.head()

(804, 2079)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,skew_90.0,skew_91.0,skew_92.0,skew_93.0,skew_94.0,skew_95.0,skew_96.0,skew_97.0,skew_98.0,skew_99.0
0,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,1.483669,0.931814,1.783425,1.402618,1.642910,1.662415,1.193939,0.724341,0.894749,0.881159
1,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,1.250204,1.448043,0.845672,1.242082,0.901520,1.036739,1.369525,1.036283,1.513080,1.318514
2,0.0,0.029256,0.040753,0.037022,0.041347,0.035408,0.036785,0.033979,0.023970,0.023692,...,0.405441,0.597184,0.375015,0.664146,0.973991,0.429463,0.166258,0.618595,0.579454,0.639973
3,0.0,0.000261,0.000842,0.001313,0.001544,0.001405,0.002993,0.001424,0.000768,0.000992,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.003460,0.003929,0.003286,0.002893,0.010054,0.003877,0.001918,0.000824,0.000323,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## `combo_maxabund_peaks_slope_mzstats`

In [53]:
# ===== TRAIN SAMPLE =====
combo_maxabund_peaks_slope = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope.csv'))
combo_maxabund_peaks_slope_mzstats = pd.concat([combo_maxabund_peaks_slope, mz_statistics], 
                               axis = 1, 
                               ignore_index=False)

combo_maxabund_peaks_slope_mzstats = combo_maxabund_peaks_slope_mzstats.fillna(0)
combo_maxabund_peaks_slope_mzstats.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_slope_mzstats.csv'), 
                                  index=False)
print(combo_maxabund_peaks_slope_mzstats.shape)
combo_maxabund_peaks_slope_mzstats.head()

(766, 2476)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,skew_90.0,skew_91.0,skew_92.0,skew_93.0,skew_94.0,skew_95.0,skew_96.0,skew_97.0,skew_98.0,skew_99.0
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# ===== TRAIN & VALID SAMPLE (_trvl) =====
combo_maxabund_peaks_slope_trvl = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope_trvl.csv'))
combo_maxabund_peaks_slope_mzstats_trvl = pd.concat([combo_maxabund_peaks_slope_trvl, mz_statistics_trvl], 
                               axis = 1, 
                               ignore_index=False)

combo_maxabund_peaks_slope_mzstats_trvl = combo_maxabund_peaks_slope_mzstats_trvl.fillna(0)
combo_maxabund_peaks_slope_mzstats_trvl.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_slope_mzstats_trvl.csv'), 
                                  index=False)
print(combo_maxabund_peaks_slope_mzstats_trvl.shape)
combo_maxabund_peaks_slope_mzstats_trvl.head()

(1059, 2476)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,skew_90.0,skew_91.0,skew_92.0,skew_93.0,skew_94.0,skew_95.0,skew_96.0,skew_97.0,skew_98.0,skew_99.0
0,0.0,0.004085,0.004641,0.001394,0.000188,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.002270,0.002495,0.001688,0.000636,0.000597,0.000819,0.000155,0.000235,0.000227,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001336,0.002464,0.001099,0.000992,0.000676,0.000883,0.000892,0.000631,0.000361,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.005993,0.012429,0.009380,0.005099,0.006921,0.001966,0.000499,0.000880,0.000767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# ===== VALID & TEST SAMPLE (_VT) =====
combo_maxabund_peaks_slope_VT = pd.read_csv(os.path.join(config.DATA_DIR_OUT, 'combo_maxabund_peaks_slope_VT.csv'))
combo_maxabund_peaks_slope_mzstats_VT = pd.concat([combo_maxabund_peaks_slope_VT, combo_maxabund_peaks_slope_VT], 
                               axis = 1, 
                               ignore_index=False)

combo_maxabund_peaks_slope_mzstats_VT = combo_maxabund_peaks_slope_mzstats_VT.fillna(0)
combo_maxabund_peaks_slope_mzstats_VT.to_csv(os.path.join(config.DATA_DIR_OUT + 
                                               'combo_maxabund_peaks_slope_mzstats_VT.csv'), 
                                  index=False)
print(combo_maxabund_peaks_slope_mzstats_VT.shape)
combo_maxabund_peaks_slope_mzstats_VT.head()

(804, 3962)


,Ion_0.0_-100_0,Ion_0.0_0_100,Ion_0.0_100_200,Ion_0.0_200_300,Ion_0.0_300_400,Ion_0.0_400_500,Ion_0.0_500_600,Ion_0.0_600_700,Ion_0.0_700_800,Ion_0.0_800_900,...,peak_abund_91.0,peak_abund_92.0,peak_abund_93.0,peak_abund_94.0,peak_abund_95.0,peak_abund_96.0,peak_abund_97.0,peak_abund_98.0,peak_abund_99.0,slope_tt
0,0.0,0.410100,0.593334,0.606578,0.678006,0.553591,0.596561,0.606902,0.535487,0.696420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.058563,0.073878,0.081069,0.083869,0.085114,0.087810,0.078234,0.079305,0.074327,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.029256,0.040753,0.037022,0.041347,0.035408,0.036785,0.033979,0.023970,0.023692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000261,0.000842,0.001313,0.001544,0.001405,0.002993,0.001424,0.000768,0.000992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.003460,0.003929,0.003286,0.002893,0.010054,0.003877,0.001918,0.000824,0.000323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
